<img src="kiitLogo.png" alt="KIIT Logo" style="float:left; padding-right:10px; width: 150px;">

<h1 align="center"><a href="https://kiit.ac.in/" style="text-decoration:none; color:black; font-size:38px;">Kalinga Institute of Industrial Technology</a></h1>




<h1 align="center"><a href="https://example.com" style="text-decoration:none; color:black;">Exploring Dimensionality Reduction with Autoencoders</a></h1>

<p align="center" style="text-align: center;">by <a href="https://www.linkedin.com/in/vishal-singh/">Vishal Singh</a>, <a href="https://www.linkedin.com/in/vedant-kumar/">Vedant Kumar</a>, and <a href="https://www.linkedin.com/in/m-aditya/">M. Aditya</a></p>



## Introduction

In this project, we explore dimensionality reduction using autoencoders, a powerful technique in machine learning for compressing high-dimensional data into a lower-dimensional representation while retaining essential information.


## Objective

The primary objective of this project is to demonstrate the effectiveness of autoencoders for dimensionality reduction tasks. Specifically, we aim to:

- Understand the principles of dimensionality reduction and its importance in data preprocessing.
- Compare traditional methods like PCA with autoencoders.
- Implement an autoencoder model using TensorFlow.
- Evaluate the performance of the autoencoder on synthetic data.


##  Dimensionality Reduction

The transformation of data from a high-dimensional space into a meaningful low-dimensional space, which ideally corresponds to the intrinsic dimensionality of the original data, is referred to as **dimensionality reduction**. This transformation is important in several domains because it mitigates undesired properties of high-dimensional spaces such as the curse of dimensionality.

Traditionally, dimensionality reduction is performed using linear techniques such as **Principal Components Analysis (PCA)**, which is one of the most used statistical techniques in behavioral sciences and is a standard part of measure development.

However, assumptions required for PCA are not always satisfied. PCA assumes that the relationships between variables are linear, and all variables should be assessed on an interval or ratio level of measurement. Therefore, PCA may not always be the most appropriate method of analysis.

In contrast to the traditional linear techniques for dimensionality reduction, machine learning techniques can deal with complex nonlinear data, and they represent a valuable alternative to classical methods. Among others, **Autoencoders** seem a valuable alternative to PCA for dimensionality reduction.






## Autoencoders: A Nonlinear Approach to Dimensionality Reduction

**Autoencoder**, also called auto-associative neural network, is a multi-layer perceptron having the same number of outputs as inputs, designed to learn an approximation to the identity function, so as the output is as similar to the input as possible. This is achieved by minimizing an error function which captures the degree of mismatch between the input vectors and their reconstructions.

Autoencoder has the advantage of not being limited to linear transformations and can learn more complicated relations between visible and hidden units, although it contains standard principal component analysis as a special case. However, unlike PCA, the coordinates of the output of the bottleneck are correlated and are not sorted in descending order of variance. Moreover, computationally intensive nonlinear optimization techniques are required for training.

Autoencoders offer the flexibility to capture complex, nonlinear relationships in data, making them well-suited for dimensionality reduction tasks where linear techniques may be insufficient.


<img src="AE.png" alt="AutEncoder" width="50%" height="50%">

## Code Implementation
Let's start by importing necessary libraries and defining the autoencoder model.
### Import Required Libraries

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt


### Global Variables

In [ ]:
Batch_size = 64  # Number of samples per mini-batch
num_iterations = 200  # Number of training iterations

### Load Data


In [ ]:
# Load data from CSV
def load_data(file_path):
    """
    Function to load data from a CSV file.
    
    Parameters:
        file_path (str): Path to the CSV file.
        
    Returns:
        input_data (numpy.ndarray): Input features.
        output_data (numpy.ndarray): Output features.
    """
    data = pd.read_csv(file_path)
    # Assuming the first 5 columns are input and output features
    input_data = data.iloc[:, :5].values
    output_data = data.iloc[:, :5].values
    return input_data, output_data


### Define the Autoencoder Model


In [ ]:
class Autoencoder(tf.keras.Model):
    def __init__(self, input_dim, bottleneck_dim):
        """
        Class constructor to initialize the Autoencoder model.
        
        Parameters:
            input_dim (int): Dimensionality of input data.
            bottleneck_dim (int): Dimensionality of the bottleneck layer.
        """
        super(Autoencoder, self).__init__()
        self.input_dim = input_dim
        self.bottleneck_dim = bottleneck_dim
        
        # Encoder layers
        self.encoder_layers = []
        for i in range(input_dim - 1, bottleneck_dim - 1, -1):
            self.encoder_layers.append(tf.keras.layers.Dense(units=i, activation='relu'))
        
        # Bottleneck layer
        self.bottleneck_layer = tf.keras.layers.Dense(units=bottleneck_dim, activation='relu')
        
        # Decoder layers
        self.decoder_layers = []
        for i in range(bottleneck_dim + 1, input_dim + 1):
            self.decoder_layers.append(tf.keras.layers.Dense(units=i, activation='relu'))
        
        # Output layer
        self.output_layer = tf.keras.layers.Dense(units=input_dim, activation='sigmoid')
    
    def call(self, inputs):
        """
        Forward pass of the Autoencoder model.
        
        Parameters:
            inputs (tf.Tensor): Input data.
        
        Returns:
            reconstructed (tf.Tensor): Reconstructed output data.
        """
        # Encoder pass
        x = inputs
        for layer in self.encoder_layers:
            x = layer(x)
        
        # Bottleneck pass
        x = self.bottleneck_layer(x)
        
        # Decoder pass
        for layer in self.decoder_layers:
            x = layer(x)
        
        # Output pass
        reconstructed = self.output_layer(x)
        return reconstructed


### Function to Train the Model


In [ ]:
def train_model(autoencoder, input_data, output_data):
    """
    Function to train the Autoencoder model.
    
    Parameters:
        autoencoder (Autoencoder): Autoencoder model to train.
        input_data (numpy.ndarray): Input features.
        output_data (numpy.ndarray): Output features.
        
    Returns:
        errors (list): List of mean squared errors per iteration.
    """
    errors = []
    optimizer = tf.keras.optimizers.Adam()
    for i in range(num_iterations):
        for j in range(0, len(input_data), Batch_size):
            batch_input = input_data[j:j+Batch_size]
            batch_output = output_data[j:j+Batch_size]
            with tf.GradientTape() as tape:
                reconstructed = autoencoder(batch_input)
                loss = tf.reduce_mean(tf.square(reconstructed - batch_output))
            gradients = tape.gradient(loss, autoencoder.trainable_variables)
            optimizer.apply_gradients(zip(gradients, autoencoder.trainable_variables))
        errors.append(loss.numpy())
        print(f"Iteration {i+1}, Loss: {loss.numpy()}")
    return errors


### Function to Plot Error vs. Iteration


In [ ]:
def plot_error_vs_iteration(errors_dict):
    """
    Function to plot error vs. iteration for different bottleneck sizes.
    
    Parameters:
        errors_dict (dict): Dictionary containing errors for different bottleneck sizes.
    """
    plt.figure(figsize=(10, 6))
    for bottleneck_size, error_list in errors_dict.items():
        plt.plot(range(1, num_iterations + 1), error_list, label=f"Bottleneck Size: {bottleneck_size}")
    plt.xlabel("Iteration")
    plt.ylabel("Mean Squared Error")
    plt.title("Error vs. Iteration for Different Bottleneck Sizes")
    plt.legend()
    plt.grid(True)
    plt.show()


## Data Loading and Model Training
Now, let's load the data, define the autoencoder model, and train it on the synthetic dataset.

In [ ]:
# Load data
input_data, output_data = load_data("dataset3.csv")
    
# Define bottleneck sizes to test
bottleneck_sizes = [1, 2, 3, 4]
    
# Dictionary to store errors for different bottleneck sizes
errors_dict = {}
    
# Iterate over bottleneck sizes
for bottleneck_size in bottleneck_sizes:
    print(f"Training for Bottleneck Size: {bottleneck_size}")
    autoencoder = Autoencoder(input_dim=5, bottleneck_dim=bottleneck_size)
    autoencoder.build(input_shape=(None, 5))
    autoencoder.compile(optimizer='adam', loss='mse')
    errors = train_model(autoencoder, input_data, output_data)
    errors_dict[bottleneck_size] = errors


## Results and Analysis

### Reconstruction Performance

The autoencoder was trained on synthetic data with varying bottleneck sizes. It was observed that the reconstruction error decreased as the bottleneck size increased, suggesting that the autoencoder effectively captured essential features with a larger latent space.

### Latent Space Visualization

Visualizing the latent space produced by the bottleneck layer of the autoencoder provided deeper insights into the learned representations. By projecting the data onto a lower-dimensional space using techniques like t-SNE or PCA, clustering patterns and inherent structures in the data were explored.

### Feature Importance

Analyzing the importance of individual features in the learned representations is crucial for understanding their contributions to the reconstructed data. Examination of the weights of the encoder and decoder layers revealed the most influential features and their relevance in capturing essential information.



### Plot Error vs. Iteration
Let's now visualize the error vs. iteration for different bottleneck sizes. This will provide insights into the convergence behavior of the autoencoder during training and help us assess the impact of bottleneck size on reconstruction accuracy.

In [ ]:
# Plot error vs. iteration for different bottleneck sizes
plot_error_vs_iteration(errors_dict)


## Further Analysis

### Comparative Studies

Conducting comparative studies with other dimensionality reduction techniques, such as **t-SNE** or **UMAP**, can provide valuable insights into the relative performance and effectiveness of the autoencoder. By benchmarking against existing methods, we can assess the strengths and weaknesses of each approach and identify scenarios where the autoencoder excels.

### Hyperparameter Tuning

Fine-tuning hyperparameters such as learning rate, batch size, and network architecture can significantly impact the performance of the autoencoder. By systematically exploring different parameter settings and evaluating their effects on reconstruction accuracy and convergence speed, we can optimize the model for better performance.


## Conclusion
Our exploration of dimensionality reduction with autoencoders has demonstrated their effectiveness in capturing essential features of high-dimensional data and reducing its dimensionality while preserving important information. Through comprehensive analysis and experimentation, we have gained valuable insights into the capabilities and limitations of autoencoders for dimensionality reduction tasks. By continuing to refine and optimize our approach, we can unlock new opportunities for leveraging autoencoders in various domains, ranging from image and text analysis to anomaly detection and feature engineering.